In [ ]:
from datetime import datetime
print("The last time this cell was running: {}".format(datetime.now().strftime('%Y-%m-%d %H:%M:%S')))

The last time this cell was running: 2021-05-05 12:02:13


In [ ]:
!pip install twokenize
!pip install ijson
!pip install statistics

Needed Libraries and Packages

In [ ]:
import os
import json
from nltk.stem.wordnet import WordNetLemmatizer
import numpy as np
import ijson
from tqdm import tqdm
import pickle
from statistics import mean
from datetime import datetime
import ast

anx_cat=128
discrep_cat=134
tentat_cat=135
certain_cat=136
swear_cat=22
future_cat=15
politifact_fake_users = {}
politifact_real_users = {}
gossipcop_fake_users = {}
gossipcop_real_users = {}
retweet_counts = {} #tweetID:retweet_count #for fake/real news tweets not user timelines
like_counts = {} #tweetID:favorite_count #for fake/real news tweets not user timelines

Extract the base directory

In [ ]:
# # In case running from google colab
# try:
#     from google.colab import drive
#     drive.mount('/content/drive')
#     base_dir = 'drive/Shared drives/Profiling Fake News Spreaders'
# except:
#     base_dir = './'
base_dir = os.getcwd() + '/'
print('Base Dir:', base_dir)  

Base Dir: /home/local/ASUAD/mkarami/Documents/Motivational Factors/


Read and counts the retweets

In [ ]:
def read_retweets(dir):
    articles = os.listdir(dir)
    for article in articles:
        if os.path.isdir(dir + '/' + article):
            fin = open(dir + '/' + article + '/retweets.json')
            for line in fin:
                tweets = json.loads(line)
                for tweet_id in tweets:
                    retweet_counts[tweet_id] = len(tweets[tweet_id])

In [ ]:
dir = base_dir + 'gossipcop_real'
folders = os.listdir(dir)
print(len(folders))
for folder in folders:
    if os.path.isdir(dir + '/' + folder):
        files = os.listdir(dir + '/' + folder)
    if len(files)<5:
        print(folder)

16817


In [ ]:
dir = base_dir + 'gossipcop_fake'
folders = os.listdir(dir)
for folder in folders:
    if os.path.isdir(dir + '/' + folder):
        files = os.listdir(dir + '/' + folder)
    if len(files)<5:
        print(folder)

In [ ]:
read_retweets(base_dir + 'politifact_real')

In [ ]:
read_retweets(base_dir + 'politifact_fake')

In [ ]:
read_retweets(base_dir + 'gossipcop_real')

In [ ]:
read_retweets(base_dir + 'gossipcop_fake')


Read the articles and counts the likes

In [ ]:
def read_likes(dir):
    articles = os.listdir(dir)
    for article in articles:
        if os.path.isdir(dir + '/' + article):
            fin = open(dir + '/' + article + '/likes.json')
            for line in fin:
                tweets = json.loads(line)
                for tweet_id in tweets:
                    like_counts[tweet_id] = len(tweets[tweet_id])

In [ ]:
read_likes(base_dir + 'politifact_real')

In [ ]:
read_likes(base_dir + 'gossipcop_real')

In [ ]:
read_likes(base_dir + 'politifact_fake')

In [ ]:
read_likes(base_dir + 'gossipcop_fake')

For each user, find all his/her tweets and filter those with more than 3 tweets.

In [ ]:
def read_user_tweets(dir, users, out_file, activity_cutoff=1): # cut off users with less than activity_cutoff fake/real tweets
    articles = os.listdir(dir)
    total_tweets = 0
    local_retweet_counts = {} #user:[retweet_count_of_fake_tweets]
    local_likes_count = {} #user:[favorited_count_of_fake_tweets]
    for article in articles:
        if os.path.isdir(dir + '/' + article):
            fin = open(dir + '/' + article + '/tweets.json')
            tweets = json.loads(fin.readline(), strict=False)['tweets']
            for tweet in tweets:
                user_id = str(tweet['user_id'])
                total_tweets += 1
                users[user_id] = users.get(user_id,{'text':[]})
                users[user_id]['text'].append(tweet['text'])
                if user_id not in local_retweet_counts:
                    local_retweet_counts[user_id] = [retweet_counts[str(tweet['tweet_id'])]]
                else:
                    temp = local_retweet_counts[user_id]
                    temp.append(retweet_counts[str(tweet['tweet_id'])])
                    local_retweet_counts[user_id] = temp[:]
                
                if user_id not in local_likes_count:
                    local_likes_count[user_id] = [like_counts[str(tweet['tweet_id'])]]
                else:
                    temp = local_likes_count[user_id]
                    temp.append(like_counts[str(tweet['tweet_id'])])
                    local_likes_count[user_id] = temp[:]
    for user in users:
        users[user]['avg_fake_retweet_count'] = mean(local_retweet_counts[user])
        users[user]['avg_fake_favorite_count'] = mean(local_likes_count[user])
                
    users_filtered = {}
    for user in users:
        if len(users[user]['text']) >= activity_cutoff:
            users_filtered[user] = users[user]
    pickle.dump(users_filtered, open(out_file, 'wb'))
    print('Number of users with {} or more tweets is {}\n'.format(activity_cutoff, len(users_filtered)))
    return users_filtered

politifact_fake_users = {}
politifact_real_users = {}
gossipcop_fake_users = {}
gossipcop_real_users = {}
politifact_real_users = read_user_tweets(base_dir + 'politifact_real', politifact_real_users, base_dir + 'Filtered_date_users_with_three_tweets_or_more/politifact_real_users.pkl', 3)
gossipcop_real_users = read_user_tweets(base_dir +'gossipcop_real', gossipcop_real_users, base_dir +'/Filtered_date_users_with_three_tweets_or_more/gossipcop_real_users.pkl', 3)
politifact_fake_users = read_user_tweets(base_dir + 'politifact_fake', politifact_fake_users, base_dir + 'Filtered_date_users_with_three_tweets_or_more/politifact_fake_users.pkl', 3)
gossipcop_fake_users = read_user_tweets(base_dir +'gossipcop_fake', gossipcop_fake_users, base_dir +'/Filtered_date_users_with_three_tweets_or_more/gossipcop_fake_users.pkl', 3)

Number of users with 3 or more tweets is 12571

Number of users with 3 or more tweets is 2831

Number of users with 3 or more tweets is 6322

Number of users with 3 or more tweets is 29384



Save the info extracted from the profiles of the users

In [ ]:
def read_users_profiles(in_file, users, out_file):
    profiles_dict = {}
    fin = open(in_file)
    profiles = ijson.items(fin, 'user_profiles.item')
    for profile in tqdm(profiles):
        profiles_dict[profile['id_str']] = {'friends_count':profile['friends_count'], 'followers_count':profile['followers_count']}
    for user in tqdm(users):
        profile = profiles_dict.get(user, {'friends_count':None, 'followers_count':None})
        users[user]['friends_count'] = profile['friends_count']
        users[user]['followers_count'] = profile['followers_count']
    
    pickle.dump(profiles_dict, open(out_file, 'wb'))

read_users_profiles(base_dir + '/User profile data/gossipcop_real_complete_user_profile_meta_data.json', 
                    gossipcop_real_users, 
                    base_dir + 'Filtered_date_users_with_three_tweets_or_more/gossipcop_real_users_profile.pkl')

read_users_profiles(base_dir +'/User profile data/gossipcop_fake_complete_user_profile_meta_data.json', 
                    gossipcop_fake_users, 
                    base_dir +'/Filtered_date_users_with_three_tweets_or_more/gossipcop_fake_users_profile.pkl')

read_users_profiles(base_dir + 'User profile data/politifact_real_complete_user_profile_meta_data.json', 
                    politifact_real_users, 
                    base_dir + 'Filtered_date_users_with_three_tweets_or_more/politifact_real_users_profile.pkl')

read_users_profiles(base_dir + 'User profile data/politifact_fake_complete_user_profile_meta_data.json', 
                    politifact_fake_users, 
                    base_dir + 'Filtered_date_users_with_three_tweets_or_more/politifact_fake_users_profile.pkl')

69069it [00:02, 24843.68it/s]
100%|██████████| 2831/2831 [00:00<00:00, 596537.28it/s]
518004it [00:21, 23877.81it/s]
100%|██████████| 29384/29384 [00:00<00:00, 828457.16it/s]
688956it [00:26, 26012.57it/s]
100%|██████████| 12571/12571 [00:00<00:00, 648870.84it/s]
212736it [00:08, 25940.97it/s]
100%|██████████| 6322/6322 [00:00<00:00, 675404.73it/s]


Extract the users' timeline

In [ ]:
def read_users_timeline(in_file_timelines, out_file_tweet, out_file_retweet, out_file_favorite, out_file_avg_twt):
    timelines = {} # user:[tweets]
    retweet_counts = {} # user:avg_retweet_count
    favorite_counts = {} # user:avg_favorited_count
    avg_twt_count = {} # user:avg_twt_per_day
    for line in tqdm(open(in_file_timelines, encoding='windows-1252')):
        try:
            line = json.loads(line)
        except:
            continue
        if line is not None:
            avg_retweet_count = []
            avg_favorite_count = []
            twt_time = []
            timelines[str(line['user_id'])] = []
            if line['recent_tweets'] is not None:
                for tweet in line['recent_tweets']:
                    avg_retweet_count.append(int(tweet['retweet_count']))
                    avg_favorite_count.append(int(tweet['favorite_count']))
                    twt_time.append(datetime.strptime(tweet['created_at'], "%a %b %d %H:%M:%S +0000 %Y"))
                    timelines[str(line['user_id'])].append(tweet['text'])
            if len(avg_retweet_count) == 0:
                retweet_counts[str(line['user_id'])] = None
            else:
                retweet_counts[str(line['user_id'])] = mean(avg_retweet_count)
            if len(avg_favorite_count) == 0:
                favorite_counts[str(line['user_id'])] = None
            else:
                favorite_counts[str(line['user_id'])] = mean(avg_favorite_count)
            twt_time = sorted(twt_time)
            if len(twt_time) == 0:
                avg_twt_count[str(line['user_id'])] = None
            else:
                avg_twt_count[str(line['user_id'])] = len(twt_time)/float((twt_time[-1] - twt_time[0]).days+1)
    pickle.dump(timelines, open(out_file_tweet, 'wb'))
    pickle.dump(retweet_counts, open(out_file_retweet, 'wb'))
    pickle.dump(favorite_counts, open(out_file_favorite, 'wb'))
    pickle.dump(avg_twt_count, open(out_file_avg_twt, 'wb'))

read_users_timeline(base_dir +'fakenewsnet_user_recent_tweets', 
                    base_dir +'fakenewsnet_user_recent_tweets_text_only.pkl',
                    base_dir +'fakenewsnet_user_retweet_count.pkl',
                    base_dir +'fakenewsnet_user_favorite_count.pkl',
                    base_dir +'fakenewsnet_user_avg_twt_count_test.pkl')


50811it [09:30, 89.12it/s] 


In [ ]:
import codecs
tweets = pickle.load(open(base_dir +'fakenewsnet_user_recent_tweets_text_only.pkl', 'rb'))

In [ ]:
import csv
for user in tqdm(tweets):
    outfile = codecs.open(base_dir +'Tweets_for_LIWC/{}.txt'.format(user), 'w', encoding='utf-8')
    temp_tweets = ' '.join(tweets[user])
    temp_tweets = temp_tweets.replace('\n', ' ')
    outfile.write(temp_tweets)
    outfile.write('\n')


100%|██████████| 29038/29038 [00:02<00:00, 10863.97it/s]


In [ ]:
tweets = pickle.load(open(base_dir +'/fakenewsnet_user_recent_tweets_text_only.pkl', 'rb'))
retweet_counts = pickle.load(open(base_dir +'/fakenewsnet_user_retweet_count.pkl', 'rb'))
favorite_counts = pickle.load(open(base_dir +'/fakenewsnet_user_favorite_count.pkl', 'rb'))
avg_twt_count = pickle.load(open(base_dir +'/fakenewsnet_user_avg_twt_count.pkl', 'rb'))
null_users = 0

for user in tqdm(gossipcop_real_users):
    if user in tweets:
        gossipcop_real_users[user]['timeline'] = tweets[user]
        gossipcop_real_users[user]['avg_retweet_count'] = retweet_counts[user]
        gossipcop_real_users[user]['avg_favorite_count'] = favorite_counts[user]
        gossipcop_real_users[user]['avg_twt_count'] = avg_twt_count[user]
    else:
        null_users += 1
        gossipcop_real_users[user]['timeline'] = None
        gossipcop_real_users[user]['avg_retweet_count'] = None
        gossipcop_real_users[user]['avg_favorite_count'] = None
        gossipcop_real_users[user]['avg_twt_count'] = None
print ('Number of null users in gossipcop_real_users is {}'.format(null_users))
null_users = 0

for user in tqdm(gossipcop_fake_users):
    if user in tweets:
        gossipcop_fake_users[user]['timeline'] = tweets[user]
        gossipcop_fake_users[user]['avg_retweet_count'] = retweet_counts[user]
        gossipcop_fake_users[user]['avg_favorite_count'] = favorite_counts[user]
        gossipcop_fake_users[user]['avg_twt_count'] = avg_twt_count[user]
    else:
        null_users += 1
        gossipcop_fake_users[user]['timeline'] = None
        gossipcop_fake_users[user]['avg_retweet_count'] = None
        gossipcop_fake_users[user]['avg_favorite_count'] = None
        gossipcop_fake_users[user]['avg_twt_count'] = None

print ('Number of null users in gossipcop_fake_users is {}'.format(null_users))
null_users = 0

for user in tqdm(politifact_real_users):
    if user in tweets:
        politifact_real_users[user]['timeline'] = tweets[user]
        politifact_real_users[user]['avg_retweet_count'] = retweet_counts[user]
        politifact_real_users[user]['avg_favorite_count'] = favorite_counts[user]
        politifact_real_users[user]['avg_twt_count'] = avg_twt_count[user]
    else:
        null_users += 1
        politifact_real_users[user]['timeline'] = None
        politifact_real_users[user]['avg_retweet_count'] = None
        politifact_real_users[user]['avg_favorite_count'] = None
        politifact_real_users[user]['avg_twt_count'] = None
print ('Number of null users in politifact_real_users is {}'.format(null_users))
null_users = 0
        
        
for user in tqdm(politifact_fake_users):
    if user in tweets:
        politifact_fake_users[user]['timeline'] = tweets[user]
        politifact_fake_users[user]['avg_retweet_count'] = retweet_counts[user]
        politifact_fake_users[user]['avg_favorite_count'] = favorite_counts[user]
        politifact_fake_users[user]['avg_twt_count'] = avg_twt_count[user]
    else:
        null_users += 1
        politifact_fake_users[user]['timeline'] = None
        politifact_fake_users[user]['avg_retweet_count'] = None
        politifact_fake_users[user]['avg_favorite_count'] = None
        politifact_fake_users[user]['avg_twt_count'] = None
print('Number of null users in politifact_fake_users is {}'.format(null_users))
null_users = 0
        

100%|██████████| 6322/6322 [00:00<00:00, 308190.35it/s]

Number of null users in gossipcop_real_users is 37
Number of null users in gossipcop_fake_users is 17824
Number of null users in politifact_real_users is 60
Number of null users in politifact_fake_users is 13


Preparing LIWC

In [ ]:
# LIWC
from nltk.tokenize import TweetTokenizer
import string
def tokenize(text):
	lda_tokens = []
	tknzr = TweetTokenizer()
	tokens = tknzr.tokenize(text)
	for token in tokens:
		token.replace(string.whitespace, '')
		if len(token) == 0:
			continue
		elif not token.startswith('http') and not token.startswith('@'):
			lda_tokens.append(token.lower())
	lda_tokens = [token for token in lda_tokens if len(token) > 4]
	return lda_tokens


def load_liwc(liwc_words_file, category):
    words = []
    for line in open(liwc_words_file, 'r'):
        line = line.strip().split('\t')
        for cat in line[1:]:
            if int(cat) == category:
                words.append(line[0])
                break
    return words

def liwc_analysis(text, cat_words):

    if len(text) == 0 or (len(text) == 1 and text[0] == ''):
        return None

    count = 0
    for word in text:
        if word in cat_words:
            count += 1
            continue
        for cat_word in cat_words:
            if '*' not in cat_word:
                continue
            cat_word = cat_word[:-1]
            if word.startswith(cat_word) and len(word)-len(cat_word) < 5:
                count += 1
                break
    return float(count)/len(text)

def clean_text(text):
    text = text.lower().replace("'t",'').replace("'ll",'').replace("'ve", '').replace("'m",'').replace("'d",'').replace("'s",'').replace("'re",'')
    cleaned_text = []
    lmtzr = WordNetLemmatizer()
    for t in tokenize(text):
        remove_punc = dict((ord(char), None) for char in '!"$%&()*+,-./:;<=>?[]^_`{|}~')
        t = t.translate(remove_punc)
        if r'@' not in t and r'#' not in t and r'http' not in t and r'https' not in t and r'\u' not in t and t != 'rt' and not t.isdigit() and len(t) > 2:
            t = lmtzr.lemmatize(t)
            cleaned_text.append(t)
    return cleaned_text

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/local/ASUAD/mkarami/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Calculate the motivational features (Figure one in the paper)

In [ ]:
def calc_features(users, out_file):
    features = {}
    anx_words = load_liwc(base_dir +'LIWC2007_Words.txt', anx_cat)
    discrep_words = load_liwc(base_dir +'LIWC2007_Words.txt', discrep_cat)
    tentat_words = load_liwc(base_dir +'LIWC2007_Words.txt', tentat_cat)
    certain_words = load_liwc(base_dir +'LIWC2007_Words.txt', certain_cat)
    swear_words = load_liwc(base_dir +'LIWC2007_Words.txt', swear_cat)
    future_words = load_liwc(base_dir +'LIWC2007_Words.txt', future_cat)
    
    for user in tqdm(users):
        if users[user]['timeline'] is not None:
            text = clean_text(' '.join(users[user]['timeline']))
            features[user] = {}
            features[user]['anx'] = liwc_analysis(text, anx_words)
            features[user]['discrep'] = liwc_analysis(text, discrep_words)
            features[user]['tentat'] = liwc_analysis(text, tentat_words)
            features[user]['certain'] = liwc_analysis(text, certain_words)
            features[user]['swear'] = liwc_analysis(text, swear_words)
            features[user]['future'] = liwc_analysis(text, future_words)
            features[user]['friends'] = users[user]['friends_count']
            features[user]['followers'] = users[user]['followers_count']
            features[user]['avg_twt'] = users[user]['avg_twt_count']
            if users[user]['avg_retweet_count'] is not None:
                features[user]['rel_enhance_retweet'] = users[user]['avg_retweet_count'] - users[user]['avg_fake_retweet_count']
            else:
                features[user]['rel_enhance_retweet'] = None
            if users[user]['avg_favorite_count'] is not None:
                features[user]['rel_enhance_favorite'] = users[user]['avg_favorite_count'] - users[user]['avg_fake_favorite_count']
            else: 
                features[user]['rel_enhance_favorite'] = None
        else:
            features[user] = {}
            features[user]['anx'] = None
            features[user]['discrep'] = None
            features[user]['tentat'] = None
            features[user]['certain'] = None
            features[user]['swear'] = None
            features[user]['future'] = None
            features[user]['friends'] = users[user]['friends_count']
            features[user]['followers'] = users[user]['followers_count']
            features[user]['avg_twt'] = None
            features[user]['rel_enhance_retweet'] = None
            features[user]['rel_enhance_favorite'] = None
#     print(features)        
    pickle.dump(features, open(out_file, 'wb'))
        
calc_features(politifact_fake_users, base_dir +'/Filtered_date_users_with_three_tweets_or_more/features_politifact_fake_users.pkl')
calc_features(politifact_real_users, base_dir +'/Filtered_date_users_with_three_tweets_or_more/features_politifact_real_users.pkl')
calc_features(gossipcop_fake_users, base_dir +'/Filtered_date_users_with_three_tweets_or_more/features_gossipcop_fake_users.pkl')
calc_features(gossipcop_real_users, base_dir +'/Filtered_date_users_with_three_tweets_or_more/features_gossipcop_real_users.pkl')


100%|██████████| 2831/2831 [02:52<00:00, 16.39it/s]


T-Test Calculation 

In [ ]:
from scipy import stats
def t_test(x, y):
    print (stats.ttest_ind(x, y, nan_policy='omit'))

features_politifact_fake_users = pickle.load(open(base_dir +'/Filtered_date_users_with_three_tweets_or_more/features_politifact_fake_users.pkl', 'rb'))
features_politifact_fake_users = features_politifact_fake_users.values()
temp = list(features_politifact_fake_users)[0]
print (temp.keys())

features_politifact_fake_users = [np.array(list(x.values())).astype(float) for x in features_politifact_fake_users]

features_politifact_real_users = pickle.load(open(base_dir +'/Filtered_date_users_with_three_tweets_or_more/features_politifact_real_users.pkl', 'rb'))
features_politifact_real_users = features_politifact_real_users.values()
features_politifact_real_users = [np.array(list(x.values())).astype(float) for x in features_politifact_real_users]

t_test(features_politifact_fake_users, features_politifact_real_users)


features_gossipcop_fake_users = pickle.load(open(base_dir +'/Filtered_date_users_with_three_tweets_or_more/features_gossipcop_fake_users.pkl', 'rb'))
features_gossipcop_fake_users = features_gossipcop_fake_users.values()
temp = list(features_gossipcop_fake_users)[0]
print (temp.keys())
features_gossipcop_fake_users = [np.array(list(x.values())).astype(float) for x in features_gossipcop_fake_users]

features_gossipcop_real_users = pickle.load(open(base_dir +'/Filtered_date_users_with_three_tweets_or_more/features_gossipcop_real_users.pkl', 'rb'))
features_gossipcop_real_users = features_gossipcop_real_users.values()
features_gossipcop_real_users = [np.array(list(x.values())).astype(float) for x in features_gossipcop_real_users]

t_test(features_gossipcop_fake_users, features_gossipcop_real_users)

dict_keys(['anx', 'discrep', 'tentat', 'certain', 'swear', 'future', 'friends', 'followers', 'avg_twt', 'rel_enhance_retweet', 'rel_enhance_favorite'])
Ttest_indResult(statistic=masked_array(data=[-1.060969647424242, -16.131825477360145,
                   -22.911369348879962, -8.67267658691202,
                   -4.438442709285443, -14.660629355317466,
                   -0.26808379735538895, -2.6961300760615283,
                   -13.129227322121329, -3.7690750637307047,
                   -1.54443835428257],
             mask=[False, False, False, False, False, False, False, False,
                   False, False, False],
       fill_value=1e+20), pvalue=masked_array(data=[2.88717374e-001, 3.75856225e-058, 1.34157328e-114,
                   4.55995699e-018, 9.11292927e-006, 2.13620577e-048,
                   7.88637759e-001, 7.02127181e-003, 3.33409269e-039,
                   1.64355346e-004, 1.22499038e-001],
             mask=False,
       fill_value=1e+20))
dict_keys(['anx',

In [ ]:
def read_news_users(dir):
    vectors = {}
    text = []
    articles = os.listdir(dir)
    for article in tqdm(articles):
        vectors[article] = {'user_ids':set(), 'vector':[]}
        if os.path.isdir(dir + '/' + article):
            fin = open(dir + '/' + article + '/tweets.json')
            tweets = json.loads(fin.readline(), strict=False)['tweets']
            for tweet in tweets:
                user_id = str(tweet['user_id'])
                vectors[article]['user_ids'].add(user_id)
    return vectors #'article_id':{'user_ids':[], 'vector':[]}

politifact_real_news_vectors = read_news_users(base_dir + 'politifact_real')
politifact_fake_news_vectors = read_news_users(base_dir + 'politifact_fake')
gossipcop_real_news_vectors = read_news_users(base_dir + 'gossipcop_real')
gossipcop_fake_news_vectors = read_news_users(base_dir + 'gossipcop_fake')


100%|██████████| 6048/6048 [00:03<00:00, 1891.93it/s]


In [ ]:
anx_cat=128
discrep_cat=134
tentat_cat=135
certain_cat=136
swear_cat=22
future_cat=15

# print (liwc_analysis(a, anx_words))
def calc_features(users1, users2):
    anx_words = load_liwc(base_dir +'LIWC2007_Words.txt', anx_cat)
    discrep_words = load_liwc(base_dir +'LIWC2007_Words.txt', discrep_cat)
    tentat_words = load_liwc(base_dir +'LIWC2007_Words.txt', tentat_cat)
    certain_words = load_liwc(base_dir +'LIWC2007_Words.txt', certain_cat)
    swear_words = load_liwc(base_dir +'LIWC2007_Words.txt', swear_cat)
    future_words = load_liwc(base_dir +'LIWC2007_Words.txt', future_cat)
    for k in users1:
        twt = users1[k]['tweets']
        users1[k] = {}
        users1[k]['anx'] = liwc_analysis(twt, anx_words)
        users1[k]['discrep'] = liwc_analysis(twt, discrep_words)
        users1[k]['tentat'] = liwc_analysis(twt, tentat_words)
        users1[k]['certain'] = liwc_analysis(twt, certain_words)
        users1[k]['swear'] = liwc_analysis(twt, swear_words)
        users1[k]['future'] = liwc_analysis(twt, future_words)
    for k in users2:
        twt = users2[k]
        users2[k] = {}
        users2[k]['anx'] = liwc_analysis(twt, anx_words)
        users2[k]['discrep'] = liwc_analysis(twt, discrep_words)
        users2[k]['tentat'] = liwc_analysis(twt, tentat_words)
        users2[k]['certain'] = liwc_analysis(twt, certain_words)
        users2[k]['swear'] = liwc_analysis(twt, swear_words)
        users2[k]['future'] = liwc_analysis(twt, future_words)
    return users1, users2
        
    

BERT

In [ ]:
# Neural Net

try:
    import transformers
except:
    !pip install transformers
    import transformers

In [ ]:
!python3 -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/home/local/ASUAD/mkarami/anaconda3/lib/python3.8/site-packages/en_core_web_sm
--> /home/local/ASUAD/mkarami/.local/lib/python3.8/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
!pip install transformers

In [ ]:
print(len(politifact_fake_users))
print(len(politifact_real_users))
print(len(gossipcop_fake_users))
print(len(gossipcop_real_users))

6322
12571
29384
2831


In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


Loading BERT tokenizer...


In [ ]:
import re
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

emoticons = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3', ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])

def clean_tweets(tweet, rm_puncs=True):
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet)
    word_tokens = tweet.replace('"', '').replace('<br />', '').replace(')', '').replace('(', '').lower().split(' ')
    
    #after tweepy preprocessing the colon symbol left remain after      #removing mentions
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
    
    #replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
    
    #remove emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)
    
    #looping through conditions
    filtered_tweet = []
    
    for w in word_tokens:
        #check tokens against stop words , emoticons and punctuations
        if (w not in emoticons and not rm_puncs) or (rm_puncs and w not in string.punctuation and w not in emoticons):
            filtered_tweet.append(w)
    
    text = ' '.join(filtered_tweet)
    return text

In [ ]:
pickle.dump(politifact_fake_users, open(base_dir+'cache/'+'politifact_fake_users', 'wb'))
pickle.dump(politifact_real_users, open(base_dir+'cache/'+'politifact_real_users', 'wb'))
pickle.dump(gossipcop_fake_users, open(base_dir+'cache/'+'gossipcop_fake_users', 'wb'))
pickle.dump(gossipcop_real_users, open(base_dir+'cache/'+'gossipcop_real_users', 'wb'))

In [ ]:
politifact_users_X = []
politifact_users_y = []
gossipcop_users_X = []
gossipcop_users_y = []

gossipcop_users = {}
for users in tqdm(politifact_fake_users):
    if politifact_fake_users[users]['timeline'] is not None:
        politifact_users_X.append(clean_tweets(' '.join(politifact_fake_users[users]['timeline'])))
        politifact_users_y.append(1)
    else:
        politifact_users_X.append("")
        politifact_users_y.append(1)
        
for users in tqdm(politifact_real_users):
    if politifact_real_users[users]['timeline'] is not None:
        politifact_users_X.append(clean_tweets(' '.join(politifact_real_users[users]['timeline'])))
        politifact_users_y.append(0)
    else:
        politifact_users_X.append("")
        politifact_users_y.append(0)
    
for users in tqdm(gossipcop_fake_users):
    if gossipcop_fake_users[users]['timeline'] is not None:
        gossipcop_users_X.append(clean_tweets(' '.join(gossipcop_fake_users[users]['timeline'])))
        gossipcop_users_y.append(1)
    else:
        gossipcop_users_X.append("")
        gossipcop_users_y.append(1)
        
for users in tqdm(gossipcop_real_users):
    if gossipcop_real_users[users]['timeline'] is not None:
        gossipcop_users_X.append(clean_tweets(' '.join(gossipcop_real_users[users]['timeline'])))
        gossipcop_users_y.append(0)
    else:
        gossipcop_users_X.append("")
        gossipcop_users_y.append(0)


100%|██████████| 2831/2831 [00:04<00:00, 642.94it/s]


In [ ]:
print(len(politifact_users_X))
print(len(gossipcop_users_X))

print(len(politifact_users_y))
print(len(gossipcop_users_y))

18893
32215
18893
32215


In [ ]:
pickle.dump(politifact_users_X, open(base_dir+'cache/'+'politifact_X', 'wb'))
pickle.dump(politifact_users_y, open(base_dir+'cache/'+'politifact_y', 'wb'))
pickle.dump(gossipcop_users_X, open(base_dir+'cache/'+'gossipcop_X', 'wb'))
pickle.dump(gossipcop_users_y, open(base_dir+'cache/'+'gossipcop_y', 'wb'))



In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
import torch
def tweet_encoding(X_data, y_data):
    input_ids = []
    attention_masks = []
#     count = 0
    # For every sentence...
    for sent in tqdm(X_data):
        # `encode_plus` will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        #   (5) Pad or truncate the sentence to `max_length`
        #   (6) Create attention masks for [PAD] tokens.
        encoded_dict = tokenizer.encode_plus(
                            sent,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = 256,           # Pad & truncate all sentences.
#                             padding = True,
#                             truncate = True, 
                            pad_to_max_length = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                       )

        # Add the encoded sentence to the list.    
        input_ids.append(encoded_dict['input_ids'])

        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks.append(encoded_dict['attention_mask'])
#         count +=1
#         if count == 10: break

    # Convert the lists into tensors.
    print(len(input_ids))
    print(len(attention_masks))
    print(len(y_data))
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(y_data)
    return input_ids, attention_masks, labels


In [ ]:
input_ids_politifact_users, attention_masks_politifact_users, labels_politifact_users = tweet_encoding(politifact_users_X, politifact_users_y)


100%|██████████| 18893/18893 [15:08<00:00, 20.80it/s] 


18893
18893
18893


In [ ]:
input_ids_gossipcop_users, attention_masks_gossipcop_users, labels_gossipcop_users = tweet_encoding(gossipcop_users_X, gossipcop_users_y)

100%|██████████| 32215/32215 [10:54<00:00, 49.19it/s] 


32215
32215
32215


In [ ]:
torch.save(input_ids_politifact_users, base_dir+'cache/'+'input_ids_politifact_users')
torch.save(attention_masks_politifact_users, base_dir+'cache/'+'attention_masks_politifact_users')
torch.save(labels_politifact_users, base_dir+'cache/'+'labels_politifact_users')

torch.save(input_ids_gossipcop_users, base_dir+'cache/'+'input_ids_gossipcop_users')
torch.save(attention_masks_gossipcop_users, base_dir+'cache/'+'attention_masks_gossipcop_users')
torch.save(labels_gossipcop_users, base_dir+'cache/'+'labels_gossipcop_users')

Switch to the training files